In [2]:
!wsl wget 'https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-train.conllu'
!wsl mv ru_syntagrus-ud-train.conllu data/ru_syntagrus-ud-train.conllu
!wsl wget 'https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu'
!wsl mv ru_syntagrus-ud-dev.conllu data/ru_syntagrus-ud-dev.conllu
!wsl wget 'https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu'
!wsl mv ru_syntagrus-ud-test.conllu data/ru_syntagrus-ud-test.conllu

In [23]:
# !wsl wget 'https://s3.amazonaws.com/realworldnlpbook/data/ud-treebanks-v2.3/UD_English-EWT/en_ewt-ud-train.conllu'
# !wsl mv en_ewt-ud-train.conllu data/en_ewt-ud-train.conllu
# !wsl wget 'https://s3.amazonaws.com/realworldnlpbook/data/ud-treebanks-v2.3/UD_English-EWT/en_ewt-ud-dev.conllu'
# !wsl mv en_ewt-ud-dev.conllu data/en_ewt-ud-dev.conllu
# !wsl wget 'https://s3.amazonaws.com/realworldnlpbook/data/ud-treebanks-v2.3/UD_English-EWT/en_ewt-ud-test.conllu'
# !wsl mv en_ewt-ud-test.conllu data/en_ewt-ud-test.conllu

In [41]:
import random

In [48]:
max_len = 0
# for file in ["train", "dev", "test"]:
file = 'train'
word2lemma = dict()
with open(f'data/ru_syntagrus-ud-{file}.conllu') as f:
    data = f.readlines()

for line in data:
    if line[0] == '#':
        continue
    if line[0] == '\n':
#             max_len = max(max_len, max(len(left), len(right)))
#             if len(left) < 100:
#                 left_sent = " ".join(left).replace("\'", " ").replace("\"", " ")
#                 right_sent = " ".join(right)
#                 outlines.append(f"{left_sent}\t{right_sent}\n")
#             left, right = [], []
        continue
    else:
        arr = line.split('\t')
        if len(arr[1].strip()) < 5 or len(arr[1]) > 20:
            continue
        if arr[1] not in word2lemma:
            max_len=max(max_len, len(arr[1]))
            word2lemma[arr[1]] = arr[2]


word_list = list(word2lemma.items())
startes = {
    'train' : 0,
    'dev' : 20000,
    'test' : 22000
}
random.shuffle(word_list)
for file in ["train", "dev", "test"]:
    outlines = []
    if file == "train":
        limit = 20000
    else:
        limit = 2000
    start = startes[file]
    print(start)
    for key, value in word_list[start:start+limit]:
        outlines.append(f"{key}\t{value}\n")
    with open(f'data/{file}.txt', 'w') as f:
        f.writelines(outlines)

0
20000
22000


In [ ]:
!allennlp train -s log_300 simple_tagger_pos.json

In [1]:
from allennlp.models.archival import load_archive

In [2]:
from src.mySeq2Seq import Seq2SeqEncoder

In [3]:
mymodel = load_archive('log_300/model.tar.gz', 0)

_jsonnet not loaded, treating C:\Users\allok\AppData\Local\Temp\tmpyxw_wlz2\config.json as json
C:\Users\allok\anaconda3\envs\nlp\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [4]:
mymodel.model.eval()

SimpleSeq2Seq(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): PytorchSeq2SeqWrapper(
    (_module): LSTM(20, 100, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (_attention): LinearAttention()
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(220, 200)
  (_output_projection_layer): Linear(in_features=200, out_features=143, bias=True)
)

In [5]:
model_reader = mymodel.config["dataset_reader"]

In [6]:
model_reader.keys()

KeysView(<allennlp.common.params.Params object at 0x000002854CF1CDD8>)

In [7]:
from allennlp.data import DatasetReader

In [8]:
from tqdm import tqdm as tqdm

In [9]:
dataset_reader_params = mymodel.config["dataset_reader"]
dataset_reader = DatasetReader.from_params(dataset_reader_params)

In [10]:
test_data = dataset_reader.read('data/test.txt')

2000it [00:00, 20619.14it/s]


In [11]:
total = 0
correct = 0
for i in tqdm(test_data):
    gold = (''.join([ str(t) for t in i["target_tokens"][1:-1]]))
    pred = ''.join([str(t) for t in mymodel.model.forward_on_instance(i)["predicted_tokens"]])
    if gold == pred:
        correct += 1
    total += 1
print(float(correct) / total)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 44.25it/s]

0.5725


In [12]:
''.join([str(t) for t in mymodel.model.forward_on_instance(dataset_reader.text_to_instance("отрицала"))["predicted_tokens"]])

'отрицать'